# Why Functional Programming

"with few exceptions, functional programming allows you to write more concise and efficient code... Python's support for functional programming is extensive" - J. Danjou (Serious Python)


## Benefits of Functional Programming

- Modularity
- Concurrency
- Testability


## Pure functions

In [1]:
def remove_last_item(a_list):
    a_list.pop(-1)
    return a_list
    
def all_but_last(a_list):
    return a_list[:-1]

print('a')
a = [1, 2, 3]
print(remove_last_item(remove_last_item(a)))
print(a, "\t\tThe list has been modified in-place, there was a 'side-effect'")

print('\nb')
b = [1, 2, 3]
print(all_but_last(all_but_last(b)))
print(b, "\tThe list was copied, no side-effect")

a
[1]
[1] 		The list has been modified in-place, there was a 'side-effect'

b
[1]
[1, 2, 3] 	The list was copied, no side-effect


## Some useful tools

### Apply functions to Items with map()

In [2]:
list(map(lambda x: x.lower(), ['A', 'B', 'C']))

['a', 'b', 'c']

### Filter lists with filter()

In [3]:
list(filter(lambda x: x.isnumeric(), ['a', '1', 'b', '2', 'c', '3']))

['1', '2', '3']

### Reduce / Inject / Fold(r)

In [4]:
from functools import reduce

reduce(lambda x, y: x + y, [1, 2, 3], 0)

6

### Reduce function arity

In [5]:
from functools import partial

def add(a, b):
    return a + b

def mul(a, b):
    return a * b

add_one = partial(add, 1)
add_two = partial(add, 2)
add_three = partial(add, 3)

double = partial(mul, 2)

print(add_one(1))
print(add_two(1))
print(add_three(1))

2
3
4


### Itertools has some interesting functions

In [6]:
from itertools import accumulate

list(accumulate([1, 2, 3], lambda x, y: x + y))

[1, 3, 6]

In [7]:
from itertools import takewhile

"".join(takewhile(lambda x: x.isalpha(), "Tuesday, 29th March"))

'Tuesday'

## Applied to our examples

In [8]:
from functools import reduce, partial


def plus(a, b): return a + b


def foldr(func, initial, seq):
    """foldr is the modular 'glue' we are using.
    
    We could just do foldr = reduce, but this makes the partial easier
    
    foldr is fold-right so we are traversing seq from right-to-left,
    so reverse the func
    """
    return reduce(func, reversed(seq), initial)

sum = partial(foldr, plus, 0)  # sum = foldr (+) 0

sum([2, 4, 6])

12

In [9]:
def times(a, b): return a * b


product = partial(foldr, times, 1)  # product = foldr (*) 1

product([2, 4, 6])

48

In [10]:
def or_(a,b): return a or b


anytrue = partial(foldr, or_, False)  # anytrue = foldr (∨) False

print(anytrue([False, False, True]))
print(anytrue([False, False, False]))

True
False


In [11]:
def and_(a, b): return a and b


alltrue = partial(foldr, and_, True)  # alltrue = foldr (∧) True

print(alltrue([True, True, True]))
print(alltrue([True, True, False]))

True
False


In [12]:
from cons import Cons


a_list = Cons.from_iter([2, 4, 6])
a_list

Cons 2 (Cons 4 (Cons 6 Nil))

In [13]:
print(sum(a_list))
print(product(a_list))
print(anytrue(a_list))
print(alltrue(a_list))

12
48
6
2


In [14]:
a = Cons(1, Cons(2))
b = Cons(3, Cons(4))

append = partial(foldr, Cons, b)

c = append(a)
c

Cons 1 (Cons 2 (Cons 3 (Cons 4 Nil)))

In [15]:
def count(a, b):
    return a + 1

length = partial(foldr, count, 0)

length(c)

4

In [16]:
def double_and_cons(b, a):
    return Cons(double(a), b)


copy = partial(foldr, Cons, None)
double_all = partial(foldr, double_and_cons, None)

print(double_all([1, 2, 3]))

Cons 2 (Cons 4 (Cons 6 Nil))


In [17]:
def f_and_cons(f, b, a):
    return Cons(f(a), b)


def map(f, iterable):
    return foldr(partial(f_and_cons, f), None, iterable)


list(map(add_three, [1, 2, 3]))

[4, 5, 6]

## Extending this to trees

In [18]:
from cons import Node

tree = Node(1, 2, Cons(3, 4))
tree

Node 1 2 (Cons 3 4)

In [40]:
def foldtree(f, g, tree, a):
    if tree is None:
        return a

    if isinstance(tree, Node):
        return g(
            g(
                foldtree(f, g, tree.left, a),
                tree.value
            ),
            foldtree(f, g, tree.right, a),
        )

    elif isinstance(tree, Cons):
        return g(
            foldtree(f, g, tree.other, a),
            tree.value
        )

    return tree

foldtree(add, add, tree, 0)

10